In [1]:
import sys
import os

cwd = os.getcwd()
os.chdir(cwd)
sys.path.append('tools')

import chat
import parse_data

In [8]:
system_prompt = """角色：你是一个数据分析专家。
任务: 根据用户问询在库表关系中找到回答问询需要调用的库名英文及其表英文。
思考流程：先认真阅读、理解库表关系，再根据用户问询的内容一一找到需要的库名英文及对应的表英文。
库表关系：
|库名中文|库名英文|表英文|表中文|表描述|
| 上市公司基本资料| AStockBasicInfoDB| LC_StockArchives| 公司概况| 收录上市公司的基本情况，包括：联系方式、注册信息、中介机构、行业和产品、公司证券品种及背景资料等内容。 |
| 上市公司基本资料| AStockBasicInfoDB| LC_NameChange| 公司名称更改状况| 收录公司名称历次变更情况，包括：中英文名称、中英文缩写名称、更改日期等内容。|
| 上市公司基本资料| AStockBasicInfoDB| LC_Business| 公司经营范围与行业变更| 1.收录上市公司、发债公司的经营范围（包括主营和兼营）以及涉足行业情况。 2.信息来源：公开转让说明书、董事会决议、定报、临时公告等。|
| 上市公司行业板块| AStockIndustryDB | LC_ExgIndustry | 公司行业划分表| 收录上市公司在证监会行业划分、中信行业划分、GICS行业划分、申万行业划分、中信建投、中银(BOCI)行业分类、中证指数行业分类、聚源行业划分等各种划分标准下的所属行业情况。|
| 上市公司行业板块| AStockIndustryDB | LC_ExgIndChange| 公司行业变更表| 内容说明： 本表记录上市公司从上市至今，由于主营业务变更导致的所属行业变化情况，采用同一行业分类标准，对其历史变更进行人为追溯，以便投资者进行公司数据回测，或开展行业估值、财务等数据的计算。 本表对公司所属行业的变更情况尽量参照原行业分类发布公司的披露数据，并对其新旧分类标准的不同之处加以判断，结合公司实际业务的变化，逐一进行人工比对，用最新的行业标准反映公司历史上的行业变更情况。 数据范围：A股上市公司 信息来源：公司公告、聚源整理。|
| 上市公司行业板块| AStockIndustryDB | LC_IndustryValuation| 行业估值指标| 内容说明：本表记录不同行业标准下的的衍生指标，包括行业静态市盈率、滚动市盈率、市净率、股息率等指标。 数据范围：2014-01-01至今 信息来源：聚源计算 |
| 上市公司行业板块| AStockIndustryDB | LC_IndFinIndicators| 行业财务指标表| 1.内容说明：本表存储行业衍生指标相关数据，反映不同行业分类标准下，各行业的成长能力、偿债能力、盈利能力和现金获取能力等。本表数据多采用整体法进行计算（如计算增长率时，采用（行业内所有公司的当期总值-上期总值）/上期总值，而非行业内公司增长率的算术平均值），且部分比例类指标对金融类公司不适用（流动比例、速动比例、毛利率等），该类指标未计算金融类公司。 2.数据范围：A股财报、业绩快报、股本结构、分红等数据，2014年至今。 3.信息来源：公告披露，聚源计算。|
| 上市公司行业板块| AStockIndustryDB | LC_COConcept| 概念所属公司表| 记录A股上市公司所属概念信息。|
| 上市公司行业板块| AStockIndustryDB | LC_ConceptList | 概念板块常量表| 记录A股市场中热点概念的相关信息|
| 上市公司产品供销/人力资源 | AStockOperationsDB| LC_SuppCustDetail| 公司供应商与客户| 1.内容说明：收录A股上市公司的主要供应商、客户清单，以及交易标的、交易金额等信息。 2.数据范围：2015年至今 3.信息来源：招股说明书、定报 |
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_SHTypeClassifi| 股东类型分类表| 本表记录聚源股东类型分类数据|
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_MainSHListNew| 股东名单(新)| 1.收录公司主要股东构成及持股数量比例、持股性质等明细资料，包括发行前和上市后的历次变动记录。 2.数据范围：1992-06-30至今 3.信息来源：招股说明书、上市公告书、定报、临时公告等。 |
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_SHNumber| 股东户数| 1.反映公司全体股东、A股股东、B股东、H股东、CDR股东的持股情况及其历史变动情况等。 2.指标计算公式： 1)户均持股比例=((股本/股东总户数)/股本)*100%（公式中分子分母描述同一股票类型） 2)相对上一期报告期户均持股比例变化=本报告期户均持股比例－上一报告期户均持股比例 3)户均持股数季度增长率=(本季度户均持股数量/上一季度户均持股数量-1)*100% 4)户均持股比例季度增长率=(本季度户均持股比例/上一季度户均持股比例-1)*100% 5)户均持股数半年增长率=(本报告期户均持股数量/前推两季度户均持股数量-1)*100%6)户均持股比例半年增长率 = (本报告期户均持股比例/前推两个季度户均持股比例-1)*100% 2.数据范围：1991-1-1至今 3.信息来源：招股说明书、上市公告书、定报、临时公告、深交所互动易、上证e互动等。|
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_Mshareholder| 大股东介绍| 1.收录上市公司及发债企业大股东的基本资料，包括直接持股和间接持股，以及持股比例、背景介绍等内容。 2.数据范围：2004-12-31至今 3.信息来源：募集说明书、招股说明书、定报、临时公告等|
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_ActualController| 公司实际控制人| 1.收录根据上市公司在招投说明书、定期报告、及临时公告中披露的实际控制人结构图判断的上市公司实际控制人信息。2.目前只处理实际控制人有变动的数据，下期和本期相比如无变化，则不做处理。 3.数据范围：2004-12-31至今 4.信息来源：招股说明书、上市公告书、定报、临时公告等。|
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_ShareStru| 公司股本结构变动| 1.收录上市公司股本结构历史变动情况。其中：标注“披露”的字段为公司公告原始披露，标注“计算”的字段为聚源依据股权登记日，并且考虑高管股锁定的实际情况计算所得的股本结构。 2.数据范围：1990-12-10至今 3.信息来源：招股说明书、上市公告书、定报、临时公告等。|
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_StockHoldingSt| 股东持股统计| 1.收录报告期末，各类机构投资者对每只股票的持仓情况，以及前十大（无限售条件）股东合计持股情况等。 2.机构持股统计中，基金持股综合考虑了上市公司披露的十大股东数据以及基金报告中披露的基金持股数据；机构持股合计包含上市公司披露的股东持股以及在同一截止时点上基金披露的所持股票数据。 3.计算公式： 1)机构持有无限售流通股数量=机构持有无限售流通A股之和2)机构持有无限售流通股比例=(机构持有无限售流通股数量/无限售A股)*100%3)机构持有A股数量=机构持有A股之和4)机构持有A股比例=(机构持有A股数量/A股总数)*100%5)机构持有股票数量=机构持有股票之和6)机构持有股票比例=(机构持有股票数量/总股本)*100% 4.数据范围：1992年至今 5.信息来源：招股说明书、上市公告书、定报、临时公告等|
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_ShareTransfer| 股东股权变动| 1.收录公司股东股权转让、二级市场买卖、股权拍卖、大宗交易、股东重组等引起股东股权变动方面的明细资料，并包含与股权分置改革相关的股东增持、减持等信息。 2.数据范围：1996-01-26至今 3.信息来源：上交所和深交所大宗交易公开信息、临时公告等。|
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_ShareFP | 股东股权冻结和质押| 1.收录股东股权被冻结和质押及进展情况，包括被冻结质押股东、被接受股权质押方、涉及股数以及冻结质押期限起始和截止日等内容。 2.数据范围：1999-09-30至今 3.信息来源：股权质押公告、股权冻结公告、解除质押冻结公告等。 |
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_ShareFPSta| 股东股权冻结和质押统计| 1.收录股东股权的质押冻结统计数据，包括股东股权累计冻结质押股数、累计占冻结质押方持股数比例和累计占总股本比例等情况。 2.指标计算公式： 1)累计占冻结质押方持股数比例=股东累计冻结质押股数(股)/股东持股数 2)累计占总股本比例 =股东累计冻结质押股数(股)/公司总股本 3)累计占总股本比例(计算) =股东累计冻结质押股数(股)/公司总股本 3.数据范围：2006-05-15至今 4.信息来源：股权质押公告、股权冻结公告、解除质押冻结公告等。|
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_Buyback | 股份回购| 1.介绍上市公司(包含科创板)发生股份回购的相关方案信息，包括股份类别、首次信息发布日期、回购协议签署日、股份被回购方、回购数量上限与下限、回购价格上限与下限、回购期限起始与截止日等内容。 2.数据范围：1994-06-23至今 3.信息来源：回购公告、董事会公告等。|
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_BuybackAttach| 股份回购关联表| 1.补充上市公司(包含科创板)发生股份回购的相关信息，包括本次回购数量、累计回购数量、本次回购资金和累计回购数量等内容。 2.数据范围：1994-09-27至今 3.信息来源：回购公告、董事会公告等。|
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_LegalDistribution| 法人配售与战略投资者 | 1.收录公司首次发行、增发新股、发行可转债过程中采用网下配售方式过程中，获得配售的企业、基金明细。 2.数据范围：1994-04-23至今 3.信息来源：上市公司公告 |
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_NationalStockHoldSt| A股国家队持股统计 | 1.内容说明：本表记录股市国家队成员持有A股的相关信息，包含：持有A股总数，占总股本比例，持有A股数量增减，持有A股数量增减幅度等。 2.数据范围：2003-01-01至今 3.信息来源：聚源 |
| 上市公司股东与股本/公司治理| AStockShareholderDB| CS_ForeignHoldingSt| 外资持股统计| 内容说明：境外投资者持股统计，包含持股总数、持股比例，境外投资者指QFII/RQFII/深股通/全球存托凭证跨境转换机构/全球存托凭证存托人。 数据范围：2007年至今 信息来源：深交所、上交所|
| 上市公司财务指标/财务报表/融资与分红 | AStockFinanceDB| LC_AShareSeasonedNewIssue | A股增发| 1.收录A股增发A股、B股增发A股、H股增发A股等的明细情况，包括历次增发预案、进程日期、预案有效期、发行属性、发行价区间、发行量区间、发行日期、上网发行情况、网下配售申购情况和募集资金与费用等内容。 2.数据范围：1991-08-17至今|
| 上市公司财务指标/财务报表/融资与分红 | AStockFinanceDB| LC_ASharePlacement | A股配股| 1.收录A股历次配股预案及实施进展明细，包括预案有效期、配股价格区间、配股说明书、募集资金和配股交款日等内容。 2.数据范围：1991-03-06至今 |
| 上市公司财务指标/财务报表/融资与分红 | AStockFinanceDB| LC_Dividend| 公司分红| 1.该表包括上市公司历次分红预案及实施进展，以及下年分配次数、方式等，以分红事件为维度，一次分红做一条记录。 2.数据范围：证券上市起-至今 3.信息来源：上市公司公告 |
| 上市公司财务指标/财务报表/融资与分红 | AStockFinanceDB| LC_CapitalInvest| 资金投向说明| 1.公司自有资金、通过发行新股、增发新股、配股、发行可转债、发行企业债等方式所得募集资金的项目投资情况以及运用进展和改投状况。 2.数据范围：1988-12-01至今 3.信息来源：董事会公告、招股意向书、招股说明书等 |
| 上市公司股票行情| AStockMarketQuotesDB | CS_StockCapFlowIndex| 境内股票交易资金流向指标 | 内容说明： 1、收录深沪京交易所正常交易的股票在每个交易日基于不同成交金额区间及成交时间区间主动及含主动被动交易的累计流入流出金额、量等信息衍生计算的统计类指标 2、数据提供范围说明 2023-10-09 及以后提供完整全盘、开盘、尾盘主买主卖及含主动被动数据 2022-11-15~2023-09-28 仅提供全盘主买主卖及含主动被动资金流向数据 2016-11-29~2022-11-14 仅提供全盘含主动被动资金流向数据 数据范围：2016-11-29至今 信息来源：基于交易所行情数据衍生计算|
| 上市公司股票行情| AStockMarketQuotesDB | CS_TurnoverVolTecIndex| 境内股票成交量技术指标| 内容说明：收录境内股票上市之日起基于日、周、月、季、半年、年K线行情衍生计算的成交量技术指标 数据范围：股票上市起-至今 信息来源：基于沪深京交易所及股转系统行情数据衍生计算 |
| 上市公司股票行情| AStockMarketQuotesDB | CS_StockPatterns| 股票技术形态表| 内容说明：收录股票从最近一个交易日往前追溯一段时期的行情表现和技术形态表现，包括近1周、近1月、近3月、近半年、近1年、上市以来的表现情况，以及连涨跌天数、连续放量缩量天数、向上向下有效突破均线、N天M板、均线多空头排列看涨看跌等技术形态指标。本表覆盖的证券品种有A股、B股、中国存托凭证(CDR), 覆盖的上市标志有主板、三板、创业板、科创板。数据范围：股票上市或挂牌起-至今 信息来源：基于沪深京交易所及股转系统行情数据衍生计算|
| 上市公司股票行情| AStockMarketQuotesDB | QT_DailyQuote| 日行情表| 1.收录股票、债券（不包含银行间交易的债券）、基金、指数每个交易日收盘行情数据，包括昨收盘、今开盘、最高价、最低价、收盘价、成交量、成交金额、成交笔数等行情指标。 2.数据范围：证券上市起-至今 3.信息来源：上交所/深交所/北交所每日行情收盘文件|
| 上市公司股票行情| AStockMarketQuotesDB | QT_StockPerformance| 股票行情表现(新)| 1.内容说明： 收录股票从最近一个交易日往前追溯一段时期的行情表现信息，包括近1周、1周以来、近1月、1月以来、近3月、近半年、近1年、今年以来、上市以来的表现情况，以及β、α、波动率、夏普比率等风险指标，本表包含停牌数据。 计算方法： 1)区间成交金额=∑区间每个交易日成交金额2)区间成交量=∑区间每个交易日成交量3)区间涨跌幅=(区间内最新复权收盘价/区间首日复权昨收盘－1)*1004)区间振幅=(区间最高复权价－区间最低复权家价)/区间首日复权昨收盘*1005)区间换手率=区间每一天换手率的合计值6) 区间成交均价=区间成交金额之和/区间成交量之和（考虑了区间有除权的情况）7) 区间日均成交金额=区间成交金额之和/区间实际交易天数8) 区间日均换手率=区间每日换手率之和/区间实际交易天数 2.数据范围：股票上市起-至今 3.信息来源：基于沪深京交易所行情数据衍生计算 |
| 上市公司股票行情| AStockMarketQuotesDB | LC_SuspendResumption| 停牌复牌表| 1.收录上市公司/基金/债券停牌复牌信息，如停牌日期、停牌时间、停牌原因、停牌事项说明、停牌期限、复牌日期、复牌时间、复牌事项说明等，包括盘中临时停牌。 2.数据范围：2008.04-至今 2.信息来源：上海证券交易所、深圳证券交易所、北京证券交易所|
| 上市公司财务指标/财务报表/融资与分红 | AStockFinanceDB| LC_BalanceSheetAll | 资产负债表_新会计准则| 1.反映企业依据2007年新会计准则在年报、中报、季报中披露的资产负债表数据；并依据新旧会计准则的科目对应关系，收录主要科目的历史对应数据。 2.收录同一公司在报告期末的四种财务报告，即未调整的合并报表、未调整的母公司报表、调整后的合并报表以及调整后的母公司报表。 3.若某个报告期的数据有多次调整，则该表展示历次调整数据。 4.该表中各财务科目的单位均为人民币元。 5.带“##”的特殊项目为单个公司披露的非标准化的科目，对应的“特殊字段说明”字段将对其作出说明；带“##”的调整项目是为了让报表的各个小项借贷平衡而设置的，便于客户对报表的遗漏和差错进行判断。 6.数据范围：1989-12-31至今 7.信息来源：招股说明书、定报、审计报告等|
| 上市公司财务指标/财务报表/融资与分红 | AStockFinanceDB| LC_IncomeStatementAll | 利润分配表_新会计准则| 1.反映企业依据2007年新会计准则在在年报、中报、季报中披露的利润表数据；并依据新旧会计准则的科目对应关系，收录了主要科目的历史对应数据。 2.收录同一公司在报告期末的四种财务报告，即未调整的合并报表、未调整的母公司报表、调整后的合并报表以及调整后的母公司报表。 3.若某个报告期的数据有多次调整，则该表展示历次调整数据。 4.该表中各财务科目的单位均为人民币元。 5.带“##”的特殊项目为单个公司披露的非标准化的科目，对应的“特殊字段说明”字段将对其作出说明；带“##”的调整项目是为了让报表的各个小项借贷平衡而设置的，便于客户对报表的遗漏和差错进行判断。 6.数据范围：1989-12-31至今 7.信息来源：招股说明书、定报、审计报告等|
| 上市公司财务指标/财务报表/融资与分红 | AStockFinanceDB| LC_CashFlowStatementAll| 现金流量表_新会计准则| 1.反映企业依据2007年新会计准则在年报、中报、季报中披露的现金流量表数据；并依据新旧会计准则的科目对应关系，收录了主要科目的历史对应数据。 2.收录同一公司在报告期末的四种财务报告，即未调整的合并报表、未调整的母公司报表、调整后的合并报表以及调整后的母公司报表。 3.若某个报告期的数据有多次调整，则该表展示历次调整数据。 4.该表中各财务科目的单位均为人民币元。 5.带“##”的特殊项目为单个公司披露的非标准化的科目，对应的“特殊字段说明”字段将对其作出说明；带“##”的调整项目是为了让报表的各个小项借贷平衡而设置的，便于客户对报表的遗漏和差错进行判断。 6.数据范围：1998-06-30至今 7.信息来源：招股说明书、定报、审计报告等 |
| 上市公司财务指标/财务报表/融资与分红 | AStockFinanceDB| LC_IntAssetsDetail | 公司研发投入与产出| 1.内容说明：收录上市公司研发投入相关数据，主要包括研发费用投入总额、占比，研发人员构成、占比等信息。 2.数据范围：2014年至今 3.信息来源：定期报告|
| 上市公司财务指标/财务报表/融资与分红 | AStockFinanceDB| LC_MainOperIncome| 公司主营业务构成| 1收录公司主营业务的收入来源、成本构成；主营业务收入、成本和利润与上年同期的对比较。 2.数据范围：1998-12-31至今 3.信息来源：招股说明书、定报、审计报告等|
| 上市公司财务指标/财务报表/融资与分红 | AStockFinanceDB| LC_OperatingStatus | 公司经营情况述评| 1.收录公司管理层对季度、半年度、年度经营情况的自我评价，以及其后期发展计划和预测，本表涵盖了公司招股以来的历次纪录。 2.数据范围：1997-12-31至今 3.信息来源：定期报告|
| 上市公司财务指标/财务报表/融资与分红 | AStockFinanceDB| LC_AuditOpinion| 公司历年审计意见| 1.收录中介机构对公司季度、半年度、年度经营情况的评价，区分审计单位、审计意见类型，本表涵盖了公司招股以来的历次纪录。 2.数据范围：1990-12-31至今 3.信息来源：定期报告、审计报告等|
| 上市公司产品供销/人力资源 | AStockOperationsDB| LC_Staff| 公司职工构成| 1.从技术职称、专业、文化程度、年龄等几个方面介绍公司职工构成情况。 2.数据范围：1999-12-31至今 3.信息来源：定期报告、招股说明书等|
| 上市公司产品供销/人力资源 | AStockOperationsDB| LC_RewardStat| 公司管理层报酬统计| 1.按报告期统计管理层的报酬情况，包括报酬总额、前三名董事报酬、前三名高管报酬、报酬区间统计分析等。 2.数据范围：2001-12-31至今 3.信息来源：定期报告、招股说明书等|
| 上市公司公告资讯/重大事项 | AStockEventsDB| LC_Warrant | 公司担保明细| 1.收录上市公司公告中披露的担保等重大事项，包括时间内容、最新进展、事件主体/交易对象名称、企业编号、与上市公司关联关系、担保原因等指标。 2.数据范围：2001年-至今 3.信息来源：上市公司公告|
| 上市公司公告资讯/重大事项 | AStockEventsDB| LC_Credit| 公司借贷明细| 1.收录上市公司公告中披露的公司借贷等重大事项描述，包括时间内容、时间主体、交易对象名称、借贷金额、还款金额、借贷利率、借贷期限等指标。 2.数据范围：2001年-至今 3.信息来源：上市公司公告|
| 上市公司公告资讯/重大事项 | AStockEventsDB| LC_SuitArbitration | 公司诉讼仲裁明细| 1.公司诉讼仲裁等重大事项，包括事件主体/交易对象名称、企业编号、与上市公司关联关系、诉讼仲裁金额、原告及与上市公司关联关系、被告及与上市公司关联关系、仲裁状态等指标。 2.数据范围：2001-至今 3.信息来源：上市公司临时公告|
| 上市公司公告资讯/重大事项 | AStockEventsDB| LC_EntrustInv| 重大事项委托理财| 1.公司委托贷款等重大事项，包括事件主体/交易对象名称、企业编号、与上市公司关联关系、涉及金额、委托期限、委托起始日、委托截止日等指标。 2.数据范围：2001-至今 3.信息来源：上市公司公告|
| 上市公司公告资讯/重大事项 | AStockEventsDB| LC_Regroup | 公司资产重组明细| 1.公司资产重组，如资产出售与转让、资产置换、债权债务重组等重大事项描述说明。 2.数据范围：2001-至今 3.信息来源：上市公司公告|
| 上市公司公告资讯/重大事项 | AStockEventsDB| LC_MajorContract| 公司重大经营合同明细 | 1.本表存放公司重大经营合同的事项，包括事件主体/交易对象名称、企业编号、与上市公司关联关系、合同标的、合同获得方式、涉及金额、合同起始日、合同截止日、合同期限等指标。 2.数据范围：2012-至今 3.信息来源：上市公司公告 |
| 上市公司公告资讯/重大事项 | AStockEventsDB| LC_InvestorRa| 投资者关系活动| 1.收录各调研机构对上市公司调研的详情，包括调研日期、参与单位、调研人员、调研主要内容等信息。 2.数据范围：2012-至今 3.信息来源：巨潮，上交所互动易和深交所互动易 |
| 上市公司公告资讯/重大事项 | AStockEventsDB| LC_InvestorDetail| 投资者关系活动调研明细| 1、收录参与上市公司调研活动的调研机构明细数据，包括调研单位、调研人员等指标。 2、数据范围：2016-至今 3、信息来源：交易所，上交所互动易和深交所互动易 |
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_ESOP| 员工持股计划| 1.主要记录员工持股计划当期的情况：包括相关日期、事件进程、事件说明、资金来源、资金总额、股票来源、股票规模、实施是否分期、存续期、锁定期等一些情况。 2.数据范围：2014.6-至今 3.信息来源：上市公司公告 |
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_ESOPSummary | 员工持股计划概况| 1.本表主要记录员工持股计划总体情况：包括相关日期、事件进程、事件说明、资金来源、资金总额、股票来源、股票规模等一些情况。对于一些分期实施的员工持股计划，本表记录总体计划的情况。 2.数据范围：2014.6-至今 3.信息来源：上市公司公告|
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_TransferPlan| 股东增减持计划表| 1.内容说明：收录上市公司(包含科创板)股东增持计划、减持计划、被动减持计划、不减持类别指标。 2.数据范围：2005-至今 3.信息来源：上市公司公告 |
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_SMAttendInfo| 股东大会出席信息| 1.收录股东大会召开时间，地点，类别；投票方式；见证律师事务所及经办律师；全体股东出席情况；非流通股东出席情况；流通股东出席情况。 2.数据范围：1999-1-28至今|
| 港股数据库 | HKStockDB | HK_EmployeeChange| 港股公司员工数量变动表| 1.记录港股公司员工数量的变动历史记录数据，包括信息发布日期、信息来源、生效日期、变更前员工数量、变更后员工数量等。2.数据范围：2001年至今。 3.信息来源：港交所。 |
| 港股数据库 | HKStockDB | HK_StockArchives| 港股公司概况| 1.收录港股上市公司的基础信息，包括名称、成立日期、注册地点、注册资本、公司业务、所属行业分类、主席、公司秘书、联系方式等信息。 2.信息来源：港交所等。 |
| 港股数据库 | HKStockDB | CS_HKStockPerformance | 港股行情表现| 1.内容说明： 收录股票从最近一个交易日往前追溯一段时期的行情表现信息，包括近1周、1周以来、近1月、1月以来、近3月、近半年、近1年、今年以来、上市以来的表现情况，本表包含停牌数据。 2.数据范围：2005年至今。 3.数据来源：根据港交所披露数据聚源衍生计算。|
| 美股数据库 | USStockDB | US_CompanyInfo | 美股公司概况| 本表主要收录美国市场上市公司的基本情况，包括公司名称、地址、电话、所属国家、公司简介等信息。|
| 美股数据库 | USStockDB | US_DailyQuote| 美股日行情| 1.本表收录美国市场证券的日收盘行情 2.数据范围：2000年2月至今|
| 公募基金数据库 | PublicFundDB| MF_FundArchives| 公募基金概况| 1.本表记录了基金基本情况，包括基金规模、成立日期、投资类型、管理人、托管人、存续期、历史简介等。2.历史数据：1998年3月起-至今。 3.信息来源：基金公司官网披露的产品说明书。|
| 公募基金数据库 | PublicFundDB| MF_FundProdName| 公募基金产品名称| 1.本表记录基金的交易所披露简称、集中申购简称、ETF申购赎回简称等基金相关的名称类信息。 2.历史数据：1998年3月起-至今。 3.信息来源：基金公司官网披露的产品说明书。其中，4-证监会简称处理的是资本市场电子化信息披露平台-公募基金净值日报的简称；6-公告披露简称处理的是基金产品资料概要和定报披露的简称；8-基金全称处理的是发售公告或是资本市场电子化信息披露平台-基金概况的全称，是将基金的多个份额合并的基金全称。|
| 公募基金数据库 | PublicFundDB| MF_InvestAdvisorOutline| 公募基金管理人概况| 1.本表记录了基金管理人的基本情况介绍，包括成立日期、注册资本、法人代表、联系方式、背景简介等。 2.历史数据：1998年3月起-至今。 3.信息来源：基金公司官网。 |
| 公募基金数据库 | PublicFundDB| MF_Dividend| 公募基金分红| 1.本表记录基金单次分红信息，包括分红比例、登记日、除息日等信息，以及聚源根据相关数据计算的累计分红金额、累计分红次数等数据。 2.历史数据：1998年12月起-至今。 3.信息来源：基金公司官网披露的相关临时公告。|
| 诚信数据库 | CreditDB| LC_ViolatiParty| 违规当事人处罚| 1.该表以事件+当事人+处罚为维度，记录单个事件下单个当事人的每一个处罚，包括当事人及其性质、当事人编码、开始日期、截止日期、违规类型、关联关系、关联上市公司、处罚机构编码、处罚机构、涉及金额、处罚说明等指标。 2.数据范围：2014年-至今 3.信息来源：交易所、上市公司公告、证监会等|
| 指数数据库 | IndexDB| LC_IndexBasicInfo| 指数基本情况| 1.收录了市场上主要指数的基本情况，包括指数类别、成份证券类别、发布机构、发布日期、基期基点、指数发布的币种等信息。 2.数据源：中证指数有限公司、上海证券交易所、深圳证券交易所、中央国债登记结算有限责任公司、申银万国研究所、标普道琼斯指数公司等|
| 指数数据库 | IndexDB| LC_IndexComponent| 指数成份| 1.收录了市场上主要指数的成份证券构成情况，包括成份证券的市场代码、入选日期、删除日期以及成份标志等信息。 2.该表仅收录主指数成份信息，不收录与主指数关系（Relationship）为“1-币种不同，2-分红规则不同，3-分红规则和币种都不同，4-税后分红”的衍生指数的信息。 3.历史数据：1990年12月至今 4.数据源：中证指数有限公司、上海证券交易所、深圳证券交易所、申银万国研究所等 |
| 机构数据库 | InstitutionDB | LC_InstiArchive| 机构基本资料| 1.收录市场上重要机构的基本资料情况，如证券公司、信托公司、保险公司等；包含机构名称、机构信息、联系方式、机构背景等信息. 2.数据源：国家企业信用信息公示系统等. |
| 常量库 | ConstantDB| SecuMain| 证券主表| 本表收录单个证券品种（股票、基金、债券）的代码、简称、上市交易所等基础信息。|
| 常量库 | ConstantDB| HK_SecuMain| 港股证券主表| 本表收录港股单个证券品种的简称、上市交易所等基础信息。|
| 常量库 | ConstantDB| CT_SystemConst | 系统常量表| 本表收录数据库中各种常量值的具体分类和常量名称描述。|
| 常量库 | ConstantDB| QT_TradingDayNew| 交易日表(新)| 本表收录各个市场的交易日信息，包括每个日期是否是交易日，是否周、月、季、年最后一个交易日|
| 常量库 | ConstantDB| LC_AreaCode| 国家城市代码表| 本表收录世界所有国家层面的数据信息和我国不同层级行政区域的划分信息。 |
| 机构数据库 | InstitutionDB | PS_EventStru| 事件体系指引表| 收录聚源最新制定的事件分类体系。|
| 常量库 | ConstantDB| US_SecuMain| 美股证券主表| 本表收录美国等境外市场单个证券品种的简称、上市交易所等基础信息。 |
| 机构数据库 | InstitutionDB | PS_NewsSecurity| 证券舆情表| 收录了全网披露的舆情信息涉及的相关证券，对对应的事件信息，并对相应的事件的正负面情感及情感重要性进行等级划分。|"""

query = "化工纳入过多少个子类概念？"

history = [{"role": "system", "content": system_prompt}]
message = chat.create_message(query, history=history, system_prompt=system_prompt, temperature=0.1, top_p=1, response_format='text')
usage = chat.get_token_usage(message, True)
content = chat.get_content(message, True)
history = chat.build_history(history, message=message)

{'prompt_tokens': 7753, 'completion_tokens': 269, 'total_tokens': 8022}
要回答这个问题，我们需要找到与化工行业相关的子类概念数量。根据库表关系，我们可以使用 "上市公司行业板块" 数据库中的 "LC_ExgIndustry" 表，该表记录了上市公司在证监会行业划分、中信行业划分、GICS行业划分、申万行业划分、中信建投、中银(BOCI)行业分类、中证指数行业分类、聚源行业划分等各种划分标准下的所属行业情况。

因此，我们需要查询 "LC_ExgIndustry" 表中所有与化工行业相关的子类概念数量。这需要执行一个SQL查询，但是由于我无法直接执行查询，我将提供一个可能的查询语句，你可以在数据库中执行以获取结果：

```sql
SELECT COUNT(DISTINCT 子类概念)
FROM LC_ExgIndustry
WHERE 行业名称 = '化工';
```

这个查询假设 "LC_ExgIndustry" 表中有一个字段叫做 "子类概念"，它存储了化工行业的子类概念，以及一个字段叫做 "行业名称"，它存储了行业的名称。查询将计算与化工行业名称相匹配的不同子类概念的数量。

请注意，实际的字段名称可能会有所不同，你需要根据实际的数据库结构来调整查询语句。如果你能提供数据库的具体结构或者字段名称，我可以帮助你构建一个更准确的查询语句。


In [11]:
system_prompt = """角色：你是一个资深的 sql 专家。
任务: 根据用户问询在库表关系中找到回答问询需要调用的库名英文及其表英文。
思考流程：先认真阅读、理解表字段关系，再给出写出「正确」sql query 的 template，然后给出 template 中各个变量的值。如果该值当前无法确定，请给出解决方案。
表字段关系：
|LC_Warrant|ID|ID|||
|LC_Warrant|CompanyCode|公司代码|公司代码（CompanyCode）：与“证券主表（SecuMain）”中的“公司代码（CompanyCode）”关联，得到上市公司的交易代码、简称等。|CompanyCode(CompanyCode):Associatedwiththe"CompanyCode(CompanyCode)"in"SecuritiesMainTable(SecuMain)",toobtainthetradingcode,abbreviation,etc.ofthelistedcompany.|
|LC_Warrant|InitialInfoPublDate|首次信息发布日期|||
|LC_Warrant|InfoPublDate|信息发布日期|||
|LC_Warrant|InfoSource|信息来源|||
|LC_Warrant|AnnouncementType|公告类型|公告类型(AnnouncementType)与(CT_SystemConst)表中的DM字段关联，令LB=1109，得到公告类型的具体描述：1-董事会公告，2-股东大会公告，3-监事会公告，4-公司公告，5-法律意见书，6-财务报告，7-中国证监会公告，8-交易所公告，9-中介机构公告，10-基金投资组合公告，11-回访报告，12-独立董事声明，13-债券公告，14-三板市场公司公告，15-基金公告，18-收购报告书，30-新股发行公告，31-增发发行公告，32-债券发行公告，33-基金发行公告，34-配股发行公告，40-增发股本变动公告，41-配股股本变动公告，43-债券上市公告，50-分红公告，60-期货公告，61-股权分置改革说明书，99-其它。|Theannouncementtype(AnnouncementType)isassociatedwiththeDMfieldinthe(CT_SystemConst)table,withLBsetto1109,thespecificdescriptionoftheannouncementtypeisasfollows:1-BoardofDirectorsannouncement,2-Shareholders'Meetingannouncement,3-BoardofSupervisorsannouncement,4-Companyannouncement,5-Legalopinionletter,6-Financialreport,7-ChinaSecuritiesRegulatoryCommissionannouncement,8-Stockexchangeannouncement,9-Intermediaryagencyannouncement,10-Fundinvestmentportfolioannouncement,11-Follow-upreport,12-Independentdirector'sstatement,13-Bondannouncement,14-ThirdBoardmarketcompanyannouncement,15-Fundannouncement,18-Acquisitionreport,30-Newshareissuanceannouncement,31-Additionalissuanceannouncement,32-Bondissuanceannouncement,33-Fundissuanceannouncement,34-Rightsissueannouncement,40-Additionalcapitalchangeannouncement,41-Rightsissuecapitalchangeannouncement,43-Bondlistingannouncement,50-Dividendannouncement,60-Futuresannouncement,61-Equityseparationreformprospectus,99-Other.|
|LC_Warrant|DisclosureMethod|披露方式|披露方式(DisclosureMethod)与(CT_SystemConst)表中的DM字段关联，令LB=1221，得到披露方式的具体描述：1-正常披露，2-事后披露。|Thedisclosuremethod(DisclosureMethod)isassociatedwiththeDMfieldinthe(CT_SystemConst)table,withLBsetto1221,resultinginthespecificdescriptionofthedisclosuremethod:1-NormalDisclosure,2-Post-Disclosure.|
|LC_Warrant|EventContent|事件内容|||
|LC_Warrant|ActionDesc|行为描述|行为描述(ActionDesc)：本字段因字符长度限制，有被截断的情况。担保事件的完整文字描述请查看text字段“事件内容(EventContent)”。|ActionDescription:Thisfieldmaybetruncatedduetocharacterlengthlimitations.Forthecompletetextdescriptionoftheguaranteeevent,pleaserefertothe"EventContent(EventContent)"field.|
|LC_Warrant|NewestAdvance|最新进展状态描述|||
|LC_Warrant|EventSubject|事件主体|事件主体与CT_SystemConst表中的DM字段关联，令LB=1246，得到事件主体的具体描述：1-上市公司，2-下属公司，3-公司股东，4-债券发行人|TheeventsubjectisassociatedwiththeDMfieldintheCT_SystemConsttable,withLB=1246,obtainingthespecificdescriptionoftheeventsubject:1-corporate,2subsidiary,3-shareholder,4-bondissuer.|
|LC_Warrant|EventProcedure|事件进程|事件进程与CT_SystemConst表中的DM字段关联，令LB=1059ANDDMIN(1001,1004,1007,1016,1019,1025,1028,1041,1053,3099)，得到事件进程的具体描述：1001-预案，1004-决案，1007-否决，1016-未实施终止，1019-实施中，1025-解除，1028-到期，1041-续签，1053-可能涉诉|TheeventprocessisassociatedwiththeDMfieldintheCT_SystemConsttable,letLB=1059ANDDMIN(1001,1004,1007,1016,1019,1025,1028,1041,1053,3099),thespecificdescriptionoftheeventprocessisobtained:1001-Plan,1004-Decision,1007-Rejection,1016-NotImplementedTermination,1019-InImplementation,1025-Release,1028-Expiry,1041-Renewal,1053-PotentiallyLitigious.|
|LC_Warrant|ActionWays|行为方式|行为方式与CT_SystemConst表中的DM字段关联，令LB=1063ANDDMIN(1201,1203,1204,1205,1207,1209,1299)，得到行为方式的具体描述：1201-提供担保，1203-接受担保，1204-提供反担保，1205-提供担保协议，1207-接受担保协议，1209-互保协议，1299-其他|ThebehaviorpatternisassociatedwiththeDMfieldintheCT_SystemConsttable,letLB=1063ANDDMIN(1201,1203,1204,1205,1207,1209,1299),resultinginthespecificdescriptionofthebehaviorpattern:1201-Provideguarantee,1203-Acceptguarantee,1204-Providecounter-guarantee,1205-Provideguaranteeagreement,1207-Acceptguaranteeagreement,1209-Mutualguaranteeagreement,1299-Other.|
|LC_Warrant|CurrencyUnit|货币单位|货币单位与CT_SystemConst表中的DM字段关联，令LB=1068，得到货币单位的具体描述：1000-美元，1001-美分，1002-美元(次日)，1003-美元(同日)，1100-港元，1110-印度卢比，1120-印度尼西亚卢比，1130-伊朗里亚尔，1140-波兰兹罗提，1150-匈牙利福林，1160-日本元，1161-欧洲日元(离岸)，1170-约旦第纳尔，1180-科威特第纳尔，1190-阿联酋迪拉姆，1200-亚美尼亚德拉姆，1210-澳门元，1220-马来西亚林吉特，1230-安第列斯群岛盾，1240-安哥拉宽扎，1250-尼泊尔卢比，1260-哈萨克斯坦坚戈，1270-巴基斯坦卢比，1280-阿鲁巴岛弗罗林，1290-菲律宾比索，1300-阿塞拜疆马纳特，1310-波斯尼亚马克，1320-新加坡元，1330-韩国元，1340-孟加拉塔卡，1350-百慕大元，1360-泰国铢，1370-沙特里亚尔，1380-文莱林吉特，1390-不丹努尔特鲁姆，1400-博茨瓦纳普拉，1410-白俄罗斯卢布，1420-人民币元，1430-台湾元，1440-伯利兹元，1450-南苏丹镑，1460-世界投资报告法郎，1470-佛得角埃斯库多，1480-厄立特里亚纳克法，1490-埃塞俄比亚比尔，1500-福克兰群岛镑，1510-格鲁吉亚拉里，1520-直布罗陀镑，1530-克罗地亚库纳，1540-以色列新谢克尔，1550-吉尔吉斯斯坦索姆，1560-开曼群岛元，1570-莱索托马洛蒂，1580-摩尔多瓦列伊，1590-马其顿第纳尔，1600-蒙古图格里克，1610-马拉维克瓦查，1620-梅蒂卡尔，1630-纳米比亚元，1640-巴布亚新几内亚基那，1650-塞尔维亚第纳尔，1660-圣赫勒拿群岛磅，1670-圣多美和普林西比多布拉，1680-太平洋法郎，1690-非共体法郎，1700-斯威士兰里兰吉尼，1710-塔吉克斯坦索莫尼，1720-土库曼斯坦马纳特，1730-汤加潘加，1740-乌克兰格里夫纳，1750-乌兹别克斯坦苏姆，1760-瓦努阿图瓦图，1770-萨摩亚塔拉，1780-中非金融合作法郎，1790-东加勒比元，1810-人民币(离岸)，2470-坦桑尼亚先令，3000-欧元，3010-比利时法郎，3020-丹麦克朗，3030-英镑，3031-便士，3040-德国马克，3050-法国法郎，3070-意大利里拉，3090-荷兰盾，3100-葡萄牙埃斯库多，3110-立陶宛立特，3120-西班牙比塞塔，3130-拉脱维亚立特，3140-斯洛文尼亚托拉尔，3150-奥地利先令，3160-斯洛伐克克朗，3170-爱沙尼亚克朗，3180-芬兰马克，3190-安道尔比塞塔(废弃)，3260-挪威克朗，3300-瑞典克朗，3310-瑞士法郎，3311-记帐瑞士法郎，3313-清算瑞士法郎，3430-苏联卢布(废弃)，3440-俄国卢布，3450-俄罗斯卢布，4000-巴西雷亚尔(废弃)，4100-巴西里亚尔，4290-墨西哥比索，5010-加拿大元，6010-澳大利亚元，6090-新西兰元，7101-刚果法郎，7103-尼日利亚奈拉，7105-越南盾，7107-肯尼亚先令，7109-卢森堡法郎，7111-摩洛哥迪拉姆，7113-南非兰特，7115-斯里兰卡卢比，7117-苏丹第纳尔，7119-也门里亚尔(废弃)，7120-苏丹镑，7121-爱尔兰镑，7123-土耳其里拉(废弃)，7125-捷克克朗(废弃)，...|ThecurrencyunitisassociatedwiththeDMfieldintheCT_SystemConsttable.WithLB=1068,thespecificdescriptionofthecurrencyunitis:1000-USD,1001-Cents,1002-USD(nextday),1003-USD(sameday),1100-HKD,1110-INR,1120-IDR,1130-IRR,1140-PLN,1150-HUF,1160-JPY,1161-EUR(offshore),1170-JOD,1180-KWD,1190-AED,1200-AMD,1210-MOP,1220-MYR,1230-ANG,1240-AOA,1250-NPR,1260-KZT,1270-PKR,1280-AFL,1290-PHP,1300-AZM,1310-BAM,1320-SGD,1330-KRW,1340-BDT,1350-BMD,1360-THB,1370-SAR,1380-BRN,1390-BT,1400-BWP,1410-BYR,1420-CNY,1430-TWD,1440-BZD,1450-SSP,1460-FRF,1470-CVE,1480-ERN,1490-ETB,1500-FKP,1510-GE,1520-GIP,1530-HRK,1540-ILS,1550-KGS,1560-KYD,1570-LSL,1580-MDL,1590-MKD,1600-MNT,1610-MWK,1620-MET,1630-NAD,1640-PGK,1650-RSD,1660-SHP,1670-STD,1680-PF,1690-FCFA,1700-SZL,1710-TJS,1720-TMT,1730-TTD,1740-UAH,1750-UZS,1760-VES,1770-Tala,1780-CFAF,1790-XCD,1810-CNY(offshore),2470-TZS,3000-EUR,3010-BEL,3020-DKK,3030-GBP,3031-Pence,3040-DEM,3050-FRF,3070-ITL,3090-NLG,3100-PTE,3110-LTL,3120-ESP,3130-LVL,3140-SIT,3150-ATS,3160-SK,3170-EEK,3180-FIM,3190-ADP(abolished),3260-NOK,3300-SEK,3310-CHF,3311-CHF(nostro),3313-CHF(clearing),3430-SUR(abolished),3440-RUR,3450-RUB,4000-BRL(abolished),4100-BRZ,4290-MXN,5010-CAD,6010-AUD,6090-NZD,7101-CDF,7103-NGN,7105-VND,7107-KES,7109-LUF,7111-MAD,7113-ZAR,7115-LKR,7117-SDN,7119-YER(abolished),7120-SDP,7121-IEP,7123-TRY(abolished),7125-CZK(abolished),...|
|LC_Warrant|SubjectName|事件主体名称|||
|LC_Warrant|SubjectCode|事件主体企业编号|事件主体企业编号(SubjectCode)：与“机构基本资料（LC_InstiArchive）”中的“企业编号（CompanyCode）”关联，得到相关企业的基本信息。|SubjectCode:Correlatedwiththe"CompanyCode"in"LC_InstiArchive"toobtainthebasicinformationoftherelatedenterprise.|
|LC_Warrant|SubjectAssociation|与上市公司关联关系|与上市公司关联关系(SubjectAssociation)与(CT_SystemConst)表中的DM字段关联，令LB=1036，得到与上市公司关联关系的具体描述：1-本公司，2-母公司，3-控股股东，4-非控股股东，5-兄弟企业，8-间接非控股股东，9-同一领导人、亲属关系，10-下属子公司、参股公司，11-项目合作合资方，12-其他关联关系，51-间接兄弟企业，80-间接控股股东，83-潜在控股股东，84-潜在非控股股东，86-转让前控股股东，87-转让前非控股股东，121-股权受托管理人，122-受同一方控制，999-无关联关系。|Therelationshipwithlistedcompanies(SubjectAssociation)isassociatedwiththeDMfieldinthe(CT_SystemConst)table,withLB=1036,obtainingthespecificdescriptionoftherelationshipwithlistedcompanies:1-Thiscompany,2-Parentcompany,3-Controllingshareholder,4-Non-controllingshareholder,5-Sistercompany,8-Indirectnon-controllingshareholder,9-Sameleader,familyrelationship,10-Subsidiarycompany,equityparticipationcompany,11-Projectcooperationjointventure,12-Otherrelatedrelationships,51-Indirectsistercompany,80-Indirectcontrollingshareholder,83-Potentialcontrollingshareholder,84-Potentialnon-controllingshareholder,86-Previouscontrollingshareholderbeforetransfer,87-Previousnon-controllingshareholderbeforetransfer,121-Equitytrusteemanager,122-Controlledbythesameparty,999-Norelatedrelationship.|
|LC_Warrant|ObjectName|交易对象名称|||
|LC_Warrant|ObjectCode|信息对象|交易对象企业编号(ObjectCode)：与“机构基本资料（LC_InstiArchive）”中的“企业编号（CompanyCode）”关联，得到相关企业的基本信息。|Transactionobjectenterprisecode(ObjectCode):associatedwiththe"CompanyCode(CompanyCode)"in"InstitutionBasicInformation(LC_InstiArchive)",toobtainthebasicinformationoftherelatedenterprise.|
|LC_Warrant|ObjectAssociation|与上市公司关联关系|与上市公司关联关系(ObjectAssociation)与(CT_SystemConst)表中的DM字段关联，令LB=1036，得到与上市公司关联关系的具体描述：1-本公司，2-母公司，3-控股股东，4-非控股股东，5-兄弟企业，8-间接非控股股东，9-同一领导人、亲属关系，10-下属子公司、参股公司，11-项目合作合资方，12-其他关联关系，51-间接兄弟企业，80-间接控股股东，83-潜在控股股东，84-潜在非控股股东，86-转让前控股股东，87-转让前非控股股东，121-股权受托管理人，122-受同一方控制，999-无关联关系。|Therelationshipwithlistedcompanies(ObjectAssociation)isassociatedwiththeDMfieldinthe(CT_SystemConst)table,withLB=1036,obtainingthespecificdescriptionoftherelationshipwithlistedcompanies:1-Thiscompany,2-Parentcompany,3-Controllingshareholder,4-Non-controllingshareholder,5-Sistercompany,8-Indirectnon-controllingshareholder,9-Sameleader,familyrelationship,10-Subsidiary,equityjointventure,11-Projectcooperationjointventure,12-Otherrelatedparties,51-Indirectsistercompany,80-Indirectcontrollingshareholder,83-Potentialcontrollingshareholder,84-Potentialnon-controllingshareholder,86-Previouscontrollingshareholderbeforetransfer,87-Previousnon-controllingshareholderbeforetransfer,121-Trusteeofequity,122-Controlledbythesameparty,999-Norelatedrelationship.|
|LC_Warrant|AgreementDate|协议签署日期|||
|LC_Warrant|IfEnded|是否终止|是否终止（IfEnded），该字段固定以下常量：0-否；1-是|Whethertoterminate(IfEnded),thisfieldisfixedwiththefollowingconstants:0-No;1-Yes.|
|LC_Warrant|Note|附注(如适用)|担保原因(GuaranteeReason)与(CT_SystemConst)表中的DM字段关联，令LB=1056，得到担保原因的具体描述：1001-借贷，1003-银行授信，1005-借贷计划额度，1007-承兑汇票，1009-票据贴现，1011-进出口押汇，1013-续借，1015-续授信，1017-借贷延期，1019-借贷展期，1099-其他。|TheGuaranteeReason(GuaranteeReason)isassociatedwiththeDMfieldintheCT_SystemConsttable.WhenLBequals1056,thespecificdescriptionoftheguaranteereasonisobtained:1001-Loan,1003-BankCredit,1005-LoanPlanQuota,1007-AcceptanceBill,1009-BillDiscounting,1011-ImportandExportBillofExchange,1013-RenewalofLoan,1015-RenewalofCredit,1017-LoanExtension,1019-LoanDeferral,1099-Other.|
|LC_Warrant|GuaranteeReason|担保原因|||
|LC_Warrant|FirstGuaranteeSum|首次担保金额(元)|||
|LC_Warrant|LatestGuaranteeSum|最新担保金额(元)|||
|LC_Warrant|GuarantorCompany|提供担保企业|||
|LC_Warrant|GuarantorAssociation|与上市公司关联关系|与上市公司关联关系(GuarantorAssociation)与(CT_SystemConst)表中的DM字段关联，令LB=1036，得到与上市公司关联关系的具体描述：1-本公司，2-母公司，3-控股股东，4-非控股股东，5-兄弟企业，8-间接非控股股东，9-同一领导人、亲属关系，10-下属子公司、参股公司，11-项目合作合资方，12-其他关联关系，51-间接兄弟企业，80-间接控股股东，83-潜在控股股东，84-潜在非控股股东，86-转让前控股股东，87-转让前非控股股东，121-股权受托管理人，122-受同一方控制，999-无关联关系。|Therelationshipwithlistedcompanies(GuarantorAssociation)isassociatedwiththeDMfieldinthe(CT_SystemConst)table,withLB=1036,obtainingthespecificdescriptionoftherelationshipwithlistedcompanies:1-Thiscompany,2-Parentcompany,3-Controllingshareholder,4-Non-controllingshareholder,5-Sistercompany,8-Indirectnon-controllingshareholder,9-Sameleader,familyrelationship,10-Subsidiarycompany,equityjointventure,11-Projectcooperationjointventure,12-Otherrelatedparties,51-Indirectsistercompany,80-Indirectcontrollingshareholder,83-Potentialcontrollingshareholder,84-Potentialnon-controllingshareholder,86-Previouscontrollingshareholderbeforetransfer,87-Previousnon-controllingshareholderbeforetransfer,121-Equitytrusteemanager,122-Controlledbythesameparty,999-Norelatedrelationship.|
|LC_Warrant|GuaranteeAsset|担保资产|||
|LC_Warrant|SecuredParty|接受担保企业|||
|LC_Warrant|SecuredPartyAssociation|与上市公司关联关系|与上市公司关联关系(SecuredPartyAssociation)与(CT_SystemConst)表中的DM字段关联，令LB=1036，得到与上市公司关联关系的具体描述：1-本公司，2-母公司，3-控股股东，4-非控股股东，5-兄弟企业，8-间接非控股股东，9-同一领导人、亲属关系，10-下属子公司、参股公司，11-项目合作合资方，12-其他关联关系，51-间接兄弟企业，80-间接控股股东，83-潜在控股股东，84-潜在非控股股东，86-转让前控股股东，87-转让前非控股股东，121-股权受托管理人，122-受同一方控制，999-无关联关系。|Therelationshipwithlistedcompanies(SecuredPartyAssociation)isassociatedwiththeDMfieldinthe(CT_SystemConst)table,withLBsetto1036,obtainingthespecificdescriptionoftherelationshipwithlistedcompanies:1-Thiscompany,2-Parentcompany,3-Controllingshareholder,4-Non-controllingshareholder,5-Sistercompany,8-Indirectnon-controllingshareholder,9-Sameleader,familyrelationship,10-Subsidiary,equityjointventure,11-Projectcooperationjointventure,12-Otherrelatedparties,51-Indirectsistercompany,80-Indirectcontrollingshareholder,83-Potentialcontrollingshareholder,84-Potentialnon-controllingshareholder,86-Previouscontrollingshareholderbeforetransfer,87-Previousnon-controllingshareholderbeforetransfer,121-Equitytrusteemanager,122-Controlledbythesameparty,999-Norelatedrelationship.|
|LC_Warrant|CounterGuarantor|提供反担保企业|||
|LC_Warrant|CGuarantorAssociation|与上市公司关联关系|与上市公司关联关系(CGuarantorAssociation)与(CT_SystemConst)表中的DM字段关联，令LB=1036，得到与上市公司关联关系的具体描述：1-本公司，2-母公司，3-控股股东，4-非控股股东，5-兄弟企业，8-间接非控股股东，9-同一领导人、亲属关系，10-下属子公司、参股公司，11-项目合作合资方，12-其他关联关系，51-间接兄弟企业，80-间接控股股东，83-潜在控股股东，84-潜在非控股股东，86-转让前控股股东，87-转让前非控股股东，121-股权受托管理人，122-受同一方控制，999-无关联关系。|TheCGuarantorAssociationwithlistedcompaniesisassociatedwiththeDMfieldintheCT_SystemConsttable,withLBsetto1036,resultinginthespecificdescriptionofthe关联relationship:1-Thiscompany,2-Parentcompany,3-Controllingshareholder,4-Non-controllingshareholder,5-Sistercompany,8-Indirectnon-controllingshareholder,9-Sameleader,familyrelationship,10-Subsidiary,equityjointventure,11-Projectcooperationjointventure,12-Otherassociatedrelationships,51-Indirectsistercompany,80-Indirectcontrollingshareholder,83-Potentialcontrollingshareholder,84-Potentialnon-controllingshareholder,86-Transferformercontrollingshareholder,87-Transferformernon-controllingshareholder,121-Equitytrusteemanager,122-Controlledbythesameparty,999-Noassociatedrelationship.|
|LC_Warrant|CGuaranteeAsset|反担保资产|||
|LC_Warrant|LendBank|借款银行(资金出借方)|||
|LC_Warrant|LendTerm|借款期限(月)|||
|LC_Warrant|RenewalTerm|其中：续借期限(元)|||
|LC_Warrant|LendBeginDate|借贷起始日|||
|LC_Warrant|LendEndDate|借贷截止日|||
|LC_Warrant|GuaranteeTerm|担保期限(月)|||
|LC_Warrant|RenewalGuaranteeTerm|其中：续保期限(月)|||
|LC_Warrant|GuaranteeBeginDate|担保起始日|||
|LC_Warrant|GuaranteeEndDate|担保截止日|||
|LC_Warrant|DischargeGuaranteeDate|担保解除日期|||
|LC_Warrant|DischargeGuaranteeWays|担保解除方式|担保解除方式(DischargeGuaranteeWays)与(CT_SystemConst)表中的DM字段关联，令LB=1055，得到担保解除方式的具体描述：1001-正常解除，1003-担保转移解除，1005-代部分还款解除，1007-代全额还款解除。|Thewaysofguaranteedischarge(DischargeGuaranteeWays)areassociatedwiththeDMfieldinthe(CT_SystemConst)table.WhenLBequals1055,thespecificdescriptionoftheguaranteedischargemethodsareasfollows:1001-Normaldischarge,1003-Transferofguaranteedischarge,1005-Dischargebysubstitutingpartialrepayment,1007-Dischargebysubstitutingfullrepayment.|
|LC_Warrant|IfIllegality|担保是否违规|担保是否违规(IfIllegality)与(CT_SystemConst)表中的DM字段关联，令LB=999ANDDMIN(1,2)，得到担保是否违规的具体描述：1-是，2-否。|Whethertheguaranteeisillegal(IfIllegality)isassociatedwiththeDMfieldinthe(CT_SystemConst)table,letLB=999ANDDMIN(1,2),toobtainthespecificdescriptionofwhethertheguaranteeisillegal:1-yes,2-no.|
|LC_Warrant|IfOverdue|借贷是否逾期|借贷是否逾期(IfOverdue)与(CT_SystemConst)表中的DM字段关联，令LB=999ANDDMIN(1,2)，得到借贷是否逾期的具体描述：1-是，2-否。|Whethertheloanisoverdue(IfOverdue)isassociatedwiththeDMfieldinthe(CT_SystemConst)table,letLB=999ANDDMIN(1,2),toobtainthespecificdescriptionofwhethertheloanisoverdue:1-Yes,2-No.|"""

query = "天顺风能披露了多少次担保信息？"

history = [{"role": "system", "content": system_prompt}]
message = chat.create_message(query, history=history, system_prompt=system_prompt, temperature=0.1, top_p=1, response_format='text')
usage = chat.get_token_usage(message, True)
content = chat.get_content(message, True)
history = chat.build_history(history, message=message)

{'prompt_tokens': 6443, 'completion_tokens': 308, 'total_tokens': 6751}
为了回答这个问题，我们需要查询`LC_Warrant`表中的记录数，该表包含所有的担保信息。我们将使用`COUNT`函数来统计记录数，并且由于我们需要知道天顺风能的担保信息，我们还需要使用`CompanyCode`字段来筛选出天顺风能的记录。但是，我们目前没有天顺风能的`CompanyCode`。为了解决这个问题，我们需要先查询`SecuMain`表以找到天顺风能的`CompanyCode`。

以下是查询的步骤：

1. 从`SecuMain`表中获取天顺风能的`CompanyCode`。
2. 使用获取到的`CompanyCode`在`LC_Warrant`表中查询天顺风能的担保信息记录数。

SQL查询模板如下：

```sql
-- 第一步：查询天顺风能的CompanyCode
SELECT CompanyCode
FROM SecuMain
WHERE 公司简称 = '天顺风能';

-- 第二步：使用天顺风能的CompanyCode查询LC_Warrant表中的记录数
SELECT COUNT(*)
FROM LC_Warrant
WHERE CompanyCode = '天顺风能的CompanyCode';
```

在第一步中，我们需要将`'天顺风能的CompanyCode'`替换为实际查询到的`CompanyCode`值。如果`SecuMain`表中使用的是公司全称而不是简称，那么我们需要相应地调整WHERE子句中的条件。

请注意，实际的SQL查询可能需要根据数据库的具体情况（如表名、字段名的实际使用情况）进行调整。
